# About this Script

Nowdays, reporting daily, weekly, monthly or quarterly business reports e.g. sales data, has become an important yet repetitive task. This script tackles this problem. By importing raw data from a database e.g. with a SQL command, we then can import this raw data into the script. Then the script will automatically preprocess the data, create a pivot-table, as well as a chart for the report.

###### Preparing the Environment

In [5]:
import pandas as pd
import openpyxl
from openpyxl import load_workbook
from openpyxl.styles import Font
from openpyxl.chart import BarChart, Reference
import string

data = pd.read_excel("supermarket_sales.xlsx") # importing the raw dataset
data = data[["Gender", "Product line", "Total"]] # selecting the columns we want for the report
data.head()

,Gender,Product line,Total
0,Female,Health and beauty,548.9715
1,Female,Electronic accessories,80.2200
2,Male,Home and lifestyle,340.5255
3,Male,Health and beauty,489.0480
4,Male,Sports and travel,634.3785


###### Creating an automated Report with a PivotTable and Chart

In [6]:
# Making the PivotTable
report_table = data.pivot_table(index = "Gender",
                                columns = "Product line",
                                values = "Total",
                                aggfunc = "sum").round(0)


# Exporting PivotTable to Excel file and create a workbook
report_table.to_excel("report_2021.xlsx",
                      sheet_name = "Report",
                     startrow = 4)


# Create row and column references in order to make the code work even when data is added
wb = load_workbook("report_2021.xlsx")
sheet = wb["Report"]


# Cell references (sales data from original spreadsheet)
min_column = wb.active.min_column
# print("Min Columns: " + str(min_column))
max_column = wb.active.max_column
# print("Max Columns: " + str(max_column))
min_row = wb.active.min_row
# print("Min Rows: " + str(min_rows))
max_row = wb.active.max_row
# print("Max Rows: " + str(max_rows))

'''
To automate the report, need to take minimum and maximum active column and 
row, so code we're going to write keeps working even if we add more data 
'''


# Adding some chart to make it look good ;-)
barchart = BarChart() 


# locate data and categories
data = Reference(sheet,
                 min_col = min_column+1,
                 max_col = max_column,
                 min_row = min_row,
                 max_row = max_row) 
                 #including headers

categories = Reference(sheet,
                 min_col = min_column,
                 max_col = max_column,
                 min_row = min_row+1,
                 max_row = max_row) 
                 #not including headers


# adding data and categories to barchart
barchart.add_data(data, titles_from_data = True)
barchart.set_categories(categories)


# Adding chart to workbook
sheet.add_chart(barchart, "B12")
barchart.title = "Sales by Product line"
barchart.style = 5 

wb.save("report_2021.xlsx")


# Applying excel formulas through Python
sheet["B7"] = '=SUM(B5:B6)' # creates sum in B7 from B5:B6
sheet["B7"].style = "Currency" # highlights the sum row

# Calculate sum from B to G with for loop and alphabet indezes
import string
alphabet = list(string.ascii_uppercase)
excel_alphabet = alphabet[0:max_column]
# print(excel_alphabet)

wb = load_workbook("report_2021.xlsx")
sheet = wb["Report"]

# sum in columns B-G
for i in excel_alphabet: # loops through all active columns except A column (no numeric data)
    if i != "A":
        sheet[f'{i}{max_row+1}'] = f'=SUM({i}{min_row+1}:{i}{max_row})' # same as writing sheet['B7'] = '=SUM(B5:B6)'
        sheet[f'{i}{max_row+1}'].number_format = '#,##0.00€' # gives sum cell the necessary currency with number_format

# adding total label
sheet[f'{excel_alphabet[0]}{max_row+1}'] = "Total" # adding total label in column A below max column
              
wb.save('report_2021.xlsx')

# formatting report sheet
wb = load_workbook('report_2021.xlsx')
sheet = wb['Report']
sheet['A1'] = 'Sales Report'
sheet['A2'] = '2021'
sheet['A1'].font = Font('Arial', bold=True, size=20)
sheet['A2'].font = Font('Arial', bold=True, size=10)

wb.save('report_2021.xlsx')